### In this notebook I will combine the data from 2011, 2016 and 2022 files in 1 dataset and exported to SAP_c.csv
### The auxiliary data with number of private households and number of people will be prepared and exported to FY004B_c

**Next modifications will be applied to the SAP files:**
* In order to unify counties in the files SAP2011 and SAP2016, following steps will be applied:
  1. removing all excessive words as "City", "County", "City and County" and spaces (if needed)
  2. new column "Region" will be created and each row will have appropriate according value.
      * auxiliary file with region's classifications will be used
  3. Values from "County" column will be replaced with values from "Region" column.
  4. Column "Region" will be removed
  5. Column "County" will be renamed in "Region"
* in the SAP2016 columns 'County' and 'Internet' are in different order. This will be changed in rder to have same column order in all files.
* Column 'VALUE' will be renamed in 'Households with Internet access', in order to be used as a defined column in the final DataFrame
* Column 'Internet' contain 'Total' value, that will create a duplication in the numbers in case of any aggregation, so all rows with 'Total' value need to be removed.
* In the SAP2022 Column 'NUTS 3 Region' will be renamed in "Region" in order to have unified name with other data frames
* FY004B file will be modified as follow:
    1. Counties unification
    2. Creation of 2 additional columns with values for nr of private households and nr of the people in the households


**Note:** Please check "Data overview" Jupiter Notebook for reference

In [48]:
##IMPORTING LIBRARIES
import pandas as pd
import statistics as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [132]:
# Suppress the warnings
import warnings
warnings.filterwarnings('ignore')

# EDA will be structured as:<br>
####     &nbsp;1. Handling data for auxiliary file with Regions classification<br>
####     &nbsp;2. Handling data for file with Internet Types data from 2011 year<br>
####     &nbsp;3. Handling data for file with  Internet Types data from 2016 year<br>
####     &nbsp;4. Handling data for file with  Internet Types data from 2022 year<br>
####     &nbsp;5. Creating 1 DataFrame and exporting it in the csv file<br>
####     &nbsp;6. Modification of FY004B file<br>

# 1. Handling data for file with Regions classification

In [133]:
#Creating DataFrame with Regions classification

df_reg = pd.read_csv("Regions.csv")
df_reg

,Name of region,Constituent counties,Type of area
0,Border,Cavan,Administrative county
1,NaN,Donegal,Administrative county
2,NaN,Leitrim,Administrative county
3,NaN,Louth,Administrative county
4,NaN,Monaghan,Administrative county
5,NaN,Sligo,Administrative county
6,NaN,NaN,NaN
7,Dublin,Dublin,City
8,NaN,Dún Laoghaire-Rathdown,Administrative county
9,NaN,Fingal,Administrative county


#### Observing that there are NA values in the column with Regions names and empty row after each set of counties

In [134]:
#Using 'ffill' method in the column with Regions to fill NAs with the last valid value 
df_reg["Name of region "] = df_reg["Name of region "].fillna(method='ffill')
df_reg.sample(8)

,Name of region,Constituent counties,Type of area
28,South-East,Kilkenny,Administrative county
16,Midlands,Laois,Administrative county
36,South-West,Kerry,Administrative county
2,Border,Leitrim,Administrative county
40,West,Mayo,Administrative county
24,Mid-West,North Tipperary,Administrative county
7,Dublin,Dublin,City
34,South-West,Cork,City


In [135]:
#checking nr of rows with NA values
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Name of region        42 non-null     object
 1   Constituent counties  35 non-null     object
 2    Type of area         35 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [136]:
#Removing rows with NAs and making sure that there no null values left
df_reg = df_reg.dropna()
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 0 to 41
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Name of region        35 non-null     object
 1   Constituent counties  35 non-null     object
 2    Type of area         35 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [137]:
#Cheking list of counties
df_reg["Constituent counties"].unique()

array(['Cavan', 'Donegal', 'Leitrim', 'Louth', 'Monaghan', 'Sligo',
       'Dublin', 'Dún Laoghaire-Rathdown', 'Fingal', 'South Dublin',
       'Kildare', 'Meath', 'Wicklow', 'Laois', 'Longford', 'Offaly',
       'Westmeath', 'Clare', 'Limerick', 'North Tipperary ', 'Tipperary',
       'Carlow', 'Kilkenny', 'South Tipperary ', 'Waterford', 'Wexford',
       'Cork', 'Kerry', 'Galway', 'Mayo', 'Roscommon'], dtype=object)

In [138]:
# North Tiperrary and South Tiperrary contain additional space at the end, just remove it
df_reg["Constituent counties"] = df_reg["Constituent counties"].str.replace(f'Tipperary ', 'Tipperary')

In [139]:
#final check
df_reg["Constituent counties"].unique()

array(['Cavan', 'Donegal', 'Leitrim', 'Louth', 'Monaghan', 'Sligo',
       'Dublin', 'Dún Laoghaire-Rathdown', 'Fingal', 'South Dublin',
       'Kildare', 'Meath', 'Wicklow', 'Laois', 'Longford', 'Offaly',
       'Westmeath', 'Clare', 'Limerick', 'North Tipperary', 'Tipperary',
       'Carlow', 'Kilkenny', 'South Tipperary', 'Waterford', 'Wexford',
       'Cork', 'Kerry', 'Galway', 'Mayo', 'Roscommon'], dtype=object)

# 2. Handling data for file with data from 2011 year

In [140]:
#Creating dataset with internet type data per region for 2011 year 
df_it11 = pd.read_csv("SAP2011.csv")
df_it11.head(8)

,Statistic Label,Census Year,Internet,County,UNIT,VALUE
0,Households with Internet access,2011,Broadband,Carlow County,Number,11158
1,Households with Internet access,2011,Broadband,Dublin City,Number,137669
2,Households with Internet access,2011,Broadband,South Dublin,Number,68306
3,Households with Internet access,2011,Broadband,Fingal,Number,73868
4,Households with Internet access,2011,Broadband,Dún Laoghaire-Rathdown,Number,59750
5,Households with Internet access,2011,Broadband,Kildare County,Number,50093
6,Households with Internet access,2011,Broadband,Kilkenny County,Number,19816
7,Households with Internet access,2011,Broadband,Laois County,Number,16003


In [141]:
#Checking unique values
df_it11.County.unique()

array(['Carlow County', 'Dublin City', 'South Dublin', 'Fingal',
       'Dún Laoghaire-Rathdown', 'Kildare County', 'Kilkenny County',
       'Laois County', 'Longford County', 'Louth County', 'Meath County',
       'Offaly County', 'Westmeath County', 'Wexford County',
       'Wicklow County', 'Clare County', 'Cork City', 'Cork County',
       'Kerry County', 'Limerick City', 'Limerick County',
       'North Tipperary', 'South Tipperary', 'Waterford City',
       'Waterford County', 'Galway City', 'Galway County',
       'Leitrim County', 'Mayo County', 'Roscommon County',
       'Sligo County', 'Cavan County', 'Donegal County',
       'Monaghan County'], dtype=object)

##### Counties in this file contain words "County", "City", that are not included in the main clasiification file.
##### In order to be able to use regions instead of counties, as a first step I will remove those words, as well as excessive spaces.
##### Other files might need similar modification, so this is a good reason to create a simple function that remove unnecessary words and/or simbols.

In [142]:
#Method 'replace' will be used
def remove_word(df, word):
        df['County'] = df['County'].str.replace(f' {word}', '')
        return df

In [143]:
#removing unnecessary words and making sure we have no additional spaces
df_it11 = remove_word(df_it11, "County")
df_it11 = remove_word(df_it11, "City")
df_it11 = remove_word(df_it11, " ")
df_it11.County.unique()

array(['Carlow', 'Dublin', 'South Dublin', 'Fingal',
       'Dún Laoghaire-Rathdown', 'Kildare', 'Kilkenny', 'Laois',
       'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford',
       'Wicklow', 'Clare', 'Cork', 'Kerry', 'Limerick', 'North Tipperary',
       'South Tipperary', 'Waterford', 'Galway', 'Leitrim', 'Mayo',
       'Roscommon', 'Sligo', 'Cavan', 'Donegal', 'Monaghan'], dtype=object)

In [144]:
#checking DataFrame for NAs
df_it11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Statistic Label  170 non-null    object
 1   Census Year      170 non-null    int64 
 2   Internet         170 non-null    object
 3   County           170 non-null    object
 4   UNIT             170 non-null    object
 5   VALUE            170 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 8.1+ KB


###### Data frame has no NAs and ready for further modification

Next I will created a function that create new column in current data frame. In this column each row will have according  region for the available county. <br> </br>
Rational behind of creating separate function is that similar transformation will be needed for the file from 2016

In [145]:
#column Region  will be created in the dataFrame

#Function is working with 2 dataframes as arguments:
#for eah element from column "County" in the df_it will check if there is according name in 
#"Constituent counties" column from df_reg
#and if so value from column "Name of region" will be taken for new column

def create_region_column(df_it, df_reg):
    df_it['Region'] = df_it['County'].apply(
        lambda x: df_reg.loc[df_reg['Constituent counties'] == x, 'Name of region '].iloc[0] 
        if x in df_reg['Constituent counties'].to_list() else x)
    return df_it

In [146]:
#applying fuction to current dataframe and checking if new column is created with correct values
df_it11 = create_region_column(df_it11, df_reg)
df_it11.head()

,Statistic Label,Census Year,Internet,County,UNIT,VALUE,Region
0,Households with Internet access,2011,Broadband,Carlow,Number,11158,South-East
1,Households with Internet access,2011,Broadband,Dublin,Number,137669,Dublin
2,Households with Internet access,2011,Broadband,South Dublin,Number,68306,Dublin
3,Households with Internet access,2011,Broadband,Fingal,Number,73868,Dublin
4,Households with Internet access,2011,Broadband,Dún Laoghaire-Rathdown,Number,59750,Dublin


In [147]:
#replacing the counties with regions and checking 
df_it11["County"] = df_it11["Region"]
df_it11.head()

,Statistic Label,Census Year,Internet,County,UNIT,VALUE,Region
0,Households with Internet access,2011,Broadband,South-East,Number,11158,South-East
1,Households with Internet access,2011,Broadband,Dublin,Number,137669,Dublin
2,Households with Internet access,2011,Broadband,Dublin,Number,68306,Dublin
3,Households with Internet access,2011,Broadband,Dublin,Number,73868,Dublin
4,Households with Internet access,2011,Broadband,Dublin,Number,59750,Dublin


In [148]:
#renaming column to the new name and checking
df_it11 = df_it11.rename(columns={'County': 'Name of Region'})
df_it11.head()

,Statistic Label,Census Year,Internet,Name of Region,UNIT,VALUE,Region
0,Households with Internet access,2011,Broadband,South-East,Number,11158,South-East
1,Households with Internet access,2011,Broadband,Dublin,Number,137669,Dublin
2,Households with Internet access,2011,Broadband,Dublin,Number,68306,Dublin
3,Households with Internet access,2011,Broadband,Dublin,Number,73868,Dublin
4,Households with Internet access,2011,Broadband,Dublin,Number,59750,Dublin


In [68]:
#removing last column to have initial view for the table
df_it11 = df_it11.drop(columns=["Region"])
df_it11.head()

,Statistic Label,Census Year,Internet,Name of Region,UNIT,VALUE
0,Households with Internet access,2011,Broadband,South-East,Number,11158
1,Households with Internet access,2011,Broadband,Dublin,Number,137669
2,Households with Internet access,2011,Broadband,Dublin,Number,68306
3,Households with Internet access,2011,Broadband,Dublin,Number,73868
4,Households with Internet access,2011,Broadband,Dublin,Number,59750


In [69]:
#last check of the regions[just to be completely sure]
df_it11["Name of Region"].unique()

array(['South-East', 'Dublin', 'Mid-East', 'Midlands', 'Border',
       'Mid-West', 'South-West', 'West'], dtype=object)

In [70]:
#renaming column 'VALUE' and checking the result
df_it11 = df_it11.rename(columns={'VALUE': 'Households with Internet access'})
df_it11.head()

,Statistic Label,Census Year,Internet,Name of Region,UNIT,Households with Internet access
0,Households with Internet access,2011,Broadband,South-East,Number,11158
1,Households with Internet access,2011,Broadband,Dublin,Number,137669
2,Households with Internet access,2011,Broadband,Dublin,Number,68306
3,Households with Internet access,2011,Broadband,Dublin,Number,73868
4,Households with Internet access,2011,Broadband,Dublin,Number,59750


# 3. Handling data for file with data from 2016 year

In [71]:
#Creating new DataFrame and exploring it
df_it16 = pd.read_csv("SAP2016.csv")
df_it16.head()

,Statistic Label,Census Year,County,Internet,UNIT,VALUE
0,Households with Internet access,2016,Carlow,Broadband,Number,13539
1,Households with Internet access,2016,Carlow,Other,Number,1852
2,Households with Internet access,2016,Carlow,No,Number,4432
3,Households with Internet access,2016,Carlow,Not Stated,Number,642
4,Households with Internet access,2016,Carlow,Total,Number,20465


In [72]:
#re-order columns 
df_it16 = df_it16[['Statistic Label','Census Year','Internet','County','UNIT','VALUE']]
df_it16.head()

,Statistic Label,Census Year,Internet,County,UNIT,VALUE
0,Households with Internet access,2016,Broadband,Carlow,Number,13539
1,Households with Internet access,2016,Other,Carlow,Number,1852
2,Households with Internet access,2016,No,Carlow,Number,4432
3,Households with Internet access,2016,Not Stated,Carlow,Number,642
4,Households with Internet access,2016,Total,Carlow,Number,20465


In [73]:
#checking unique values
df_it16.County.unique()

array(['Carlow', 'Cavan', 'Clare', 'Cork City', 'Cork County', 'Donegal',
       'Dublin City', 'Dún Laoghaire-Rathdown', 'Fingal', 'Galway City',
       'Galway County', 'Kerry', 'Kildare', 'Kilkenny', 'Laois',
       'Leitrim', 'Limerick City and County', 'Longford', 'Louth', 'Mayo',
       'Meath', 'Monaghan', 'Offaly', 'Roscommon', 'Sligo',
       'South Dublin', 'Tipperary', 'Waterford City and County',
       'Westmeath', 'Wexford', 'Wicklow'], dtype=object)

In [74]:
#removing all irrelevant words
df_it16 = remove_word(df_it16, "City and County")
df_it16 = remove_word(df_it16, "County")
df_it16 = remove_word(df_it16, "City")
df_it16 = remove_word(df_it16, " ")
df_it16.County.unique()

array(['Carlow', 'Cavan', 'Clare', 'Cork', 'Donegal', 'Dublin',
       'Dún Laoghaire-Rathdown', 'Fingal', 'Galway', 'Kerry', 'Kildare',
       'Kilkenny', 'Laois', 'Leitrim', 'Limerick', 'Longford', 'Louth',
       'Mayo', 'Meath', 'Monaghan', 'Offaly', 'Roscommon', 'Sligo',
       'South Dublin', 'Tipperary', 'Waterford', 'Westmeath', 'Wexford',
       'Wicklow'], dtype=object)

In [75]:
#using function for Regions
df_it16 = create_region_column(df_it16, df_reg)
df_it16.head()

,Statistic Label,Census Year,Internet,County,UNIT,VALUE,Region
0,Households with Internet access,2016,Broadband,Carlow,Number,13539,South-East
1,Households with Internet access,2016,Other,Carlow,Number,1852,South-East
2,Households with Internet access,2016,No,Carlow,Number,4432,South-East
3,Households with Internet access,2016,Not Stated,Carlow,Number,642,South-East
4,Households with Internet access,2016,Total,Carlow,Number,20465,South-East


In [76]:
#replacing the counties with regions and checking 
df_it16["County"] = df_it16["Region"]
df_it16.head()

,Statistic Label,Census Year,Internet,County,UNIT,VALUE,Region
0,Households with Internet access,2016,Broadband,South-East,Number,13539,South-East
1,Households with Internet access,2016,Other,South-East,Number,1852,South-East
2,Households with Internet access,2016,No,South-East,Number,4432,South-East
3,Households with Internet access,2016,Not Stated,South-East,Number,642,South-East
4,Households with Internet access,2016,Total,South-East,Number,20465,South-East


In [77]:
#renaming column to the new name and checking
df_it16 = df_it16.rename(columns={'County': 'Name of Region'})
df_it16.head()

,Statistic Label,Census Year,Internet,Name of Region,UNIT,VALUE,Region
0,Households with Internet access,2016,Broadband,South-East,Number,13539,South-East
1,Households with Internet access,2016,Other,South-East,Number,1852,South-East
2,Households with Internet access,2016,No,South-East,Number,4432,South-East
3,Households with Internet access,2016,Not Stated,South-East,Number,642,South-East
4,Households with Internet access,2016,Total,South-East,Number,20465,South-East


In [78]:
#removing last column to have initial view for the table

df_it16 = df_it16.drop(columns=["Region"])
df_it16.head()

,Statistic Label,Census Year,Internet,Name of Region,UNIT,VALUE
0,Households with Internet access,2016,Broadband,South-East,Number,13539
1,Households with Internet access,2016,Other,South-East,Number,1852
2,Households with Internet access,2016,No,South-East,Number,4432
3,Households with Internet access,2016,Not Stated,South-East,Number,642
4,Households with Internet access,2016,Total,South-East,Number,20465


In [79]:
#Last check for obtained data
df_it16["Name of Region"].unique()

array(['South-East', 'Border', 'Mid-West', 'South-West', 'Dublin', 'West',
       'Mid-East', 'Midlands'], dtype=object)

In [80]:
#renaming column 'VALUE' and checking the result
df_it16 = df_it16.rename(columns={'VALUE': 'Households with Internet access'})
df_it16.head()

,Statistic Label,Census Year,Internet,Name of Region,UNIT,Households with Internet access
0,Households with Internet access,2016,Broadband,South-East,Number,13539
1,Households with Internet access,2016,Other,South-East,Number,1852
2,Households with Internet access,2016,No,South-East,Number,4432
3,Households with Internet access,2016,Not Stated,South-East,Number,642
4,Households with Internet access,2016,Total,South-East,Number,20465


# 4. Handling data for file with data from 2022 year

In [81]:
#Creating new DataFrame and exploring it

df_it22 = pd.read_csv("SAP2022.csv")
df_it22.head()

,Statistic Label,Census Year,Internet,NUTS 3 Region,UNIT,VALUE
0,Households with Internet access,2022,Broadband,Ireland,Number,1457883
1,Households with Internet access,2022,Broadband,Border,Number,116928
2,Households with Internet access,2022,Broadband,West,Number,134086
3,Households with Internet access,2022,Broadband,Mid-West,Number,137622
4,Households with Internet access,2022,Broadband,South-East,Number,124415


This file already has regions names and correct order of columns, so just renaming columns with regions and Values

In [82]:
#renaming column 'NUTS 3 Region' and checking the result
df_it22 = df_it22.rename(columns={'NUTS 3 Region': 'Name of Region'})

In [83]:
#renaming column 'VALUE' and checking the result
df_it22 = df_it22.rename(columns={'VALUE': 'Households with Internet access'})
df_it22.head()

,Statistic Label,Census Year,Internet,Name of Region,UNIT,Households with Internet access
0,Households with Internet access,2022,Broadband,Ireland,Number,1457883
1,Households with Internet access,2022,Broadband,Border,Number,116928
2,Households with Internet access,2022,Broadband,West,Number,134086
3,Households with Internet access,2022,Broadband,Mid-West,Number,137622
4,Households with Internet access,2022,Broadband,South-East,Number,124415


In [84]:
#checking regions to be from the same range with other files 
df_it22["Name of Region"].unique()

array(['Ireland', 'Border', 'West', 'Mid-West', 'South-East',
       'South-West', 'Dublin', 'Mid-East', 'Midlands'], dtype=object)

# 5. Creating 1 DataFrame and exporting it in the csv file "SAP.csv"

In [85]:
#checking nr of rows in the 1st dataset
df_it11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Statistic Label                  170 non-null    object
 1   Census Year                      170 non-null    int64 
 2   Internet                         170 non-null    object
 3   Name of Region                   170 non-null    object
 4   UNIT                             170 non-null    object
 5   Households with Internet access  170 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 8.1+ KB


In [86]:
#checking nr of rows in the 2nd dataset

df_it16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Statistic Label                  155 non-null    object
 1   Census Year                      155 non-null    int64 
 2   Internet                         155 non-null    object
 3   Name of Region                   155 non-null    object
 4   UNIT                             155 non-null    object
 5   Households with Internet access  155 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 7.4+ KB


In [87]:
#checking nr of rows in the 3rd dataset

df_it22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Statistic Label                  45 non-null     object
 1   Census Year                      45 non-null     int64 
 2   Internet                         45 non-null     object
 3   Name of Region                   45 non-null     object
 4   UNIT                             45 non-null     object
 5   Households with Internet access  45 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 2.2+ KB


In [88]:
#brief calc for expected result
170+155+45

370

In [89]:
#applying concatination for all 3 DataFrames
df_it = pd.concat([df_it11, df_it16,df_it22], ignore_index=True)
df_it.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Statistic Label                  370 non-null    object
 1   Census Year                      370 non-null    int64 
 2   Internet                         370 non-null    object
 3   Name of Region                   370 non-null    object
 4   UNIT                             370 non-null    object
 5   Households with Internet access  370 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 17.5+ KB


In [90]:
#Cheking if there is no mistakes or misspellings in the regions names
df_it["Name of Region"].unique()

array(['South-East', 'Dublin', 'Mid-East', 'Midlands', 'Border',
       'Mid-West', 'South-West', 'West', 'Ireland'], dtype=object)

In [91]:
# Group the DataFrame by the all columns except Value
#Rational: to have 1 row/value for each group of categories
df_grouped = df_it.groupby(['Statistic Label', 'Census Year', 'Internet', 'Name of Region', 'UNIT'])

In [92]:
# Aggregate the data in each group by summarising values in each group
df_grouped = df_grouped.agg({'Households with Internet access': 'sum'})

In [93]:
# Reset the indexes of the grouped DataFrame
df_grouped = df_grouped.reset_index()
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Statistic Label                  125 non-null    object
 1   Census Year                      125 non-null    int64 
 2   Internet                         125 non-null    object
 3   Name of Region                   125 non-null    object
 4   UNIT                             125 non-null    object
 5   Households with Internet access  125 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 6.0+ KB


In [94]:
#selecting random rows for wider observation
df_grouped.sample(15)

,Statistic Label,Census Year,Internet,Name of Region,UNIT,Households with Internet access
90,Households with Internet access,2022,No,Dublin,Number,25732
99,Households with Internet access,2022,Not stated,Dublin,Number,48808
87,Households with Internet access,2022,Broadband,South-West,Number,208255
24,Households with Internet access,2011,Other,Border,Number,15803
17,Households with Internet access,2011,Not Stated,Dublin,Number,12359
4,Households with Internet access,2011,Broadband,Midlands,Number,55102
106,Households with Internet access,2022,Not stated,West,Number,13423
33,Households with Internet access,2011,Total,Dublin,Number,466461
54,Households with Internet access,2016,No,South-West,Number,50240
27,Households with Internet access,2011,Other,Mid-West,Number,13425


In [179]:
#Export DataFrame to a csv File
df_grouped.to_csv("SAP_c.csv", index = False)

# 6. Modification for FY004B.csv file
#### Private Households number + Number of people in household

In [162]:
#performing first brief overview and creating new data frame
df_aux = pd.read_csv("FY004B.csv")
df_aux.head()

,Statistic Label,CensusYear,County and City,UNIT,VALUE
0,Private households,2011,State,Number,1654208.0
1,Private households,2011,Carlow,Number,19436.0
2,Private households,2011,Dublin City,Number,208008.0
3,Private households,2011,Dún Laoghaire-Rathdown,Number,75819.0
4,Private households,2011,Fingal,Number,93146.0


In [163]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Statistic Label  279 non-null    object 
 1   CensusYear       279 non-null    int64  
 2   County and City  279 non-null    object 
 3   UNIT             279 non-null    object 
 4   VALUE            279 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 11.0+ KB


In [164]:
#checking data by years
df_aux.CensusYear.unique()

array([2011, 2016, 2022], dtype=int64)

In [165]:
#checking labels
df_aux['Statistic Label'].unique()

array(['Private households', 'Persons in private households',
       'Average number of persons in private households'], dtype=object)

In [166]:
#analising regional data
df_aux['County and City'].unique()

array(['State', 'Carlow', 'Dublin City', 'Dún Laoghaire-Rathdown',
       'Fingal', 'South Dublin', 'Kildare', 'Kilkenny', 'Laois',
       'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford',
       'Wicklow', 'Clare', 'Cork City and Cork County', 'Kerry',
       'Limerick City and County', 'Tipperary',
       'Waterford City and County', 'Galway City', 'Galway County',
       'Leitrim', 'Mayo', 'Roscommon', 'Sligo', 'Cavan', 'Donegal',
       'Monaghan'], dtype=object)

In [167]:
# renaming column 'County and City' in order to use created function with no errors
df_aux = df_aux.rename(columns={'County and City': 'County'})
df_aux.head()

,Statistic Label,CensusYear,County,UNIT,VALUE
0,Private households,2011,State,Number,1654208.0
1,Private households,2011,Carlow,Number,19436.0
2,Private households,2011,Dublin City,Number,208008.0
3,Private households,2011,Dún Laoghaire-Rathdown,Number,75819.0
4,Private households,2011,Fingal,Number,93146.0


**As soon as we have a couple of combinations with City, County words, the function 'remove_word' will be applied step by step.**

In [168]:
df_aux = remove_word(df_aux, "City and Cork County")
df_aux.County.unique()

array(['State', 'Carlow', 'Dublin City', 'Dún Laoghaire-Rathdown',
       'Fingal', 'South Dublin', 'Kildare', 'Kilkenny', 'Laois',
       'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford',
       'Wicklow', 'Clare', 'Cork', 'Kerry', 'Limerick City and County',
       'Tipperary', 'Waterford City and County', 'Galway City',
       'Galway County', 'Leitrim', 'Mayo', 'Roscommon', 'Sligo', 'Cavan',
       'Donegal', 'Monaghan'], dtype=object)

In [169]:
df_aux = remove_word(df_aux, "City and County")
df_aux.County.unique()

array(['State', 'Carlow', 'Dublin City', 'Dún Laoghaire-Rathdown',
       'Fingal', 'South Dublin', 'Kildare', 'Kilkenny', 'Laois',
       'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford',
       'Wicklow', 'Clare', 'Cork', 'Kerry', 'Limerick', 'Tipperary',
       'Waterford', 'Galway City', 'Galway County', 'Leitrim', 'Mayo',
       'Roscommon', 'Sligo', 'Cavan', 'Donegal', 'Monaghan'], dtype=object)

In [170]:
df_aux = remove_word(df_aux, "City")
df_aux = remove_word(df_aux, "County")
df_aux.County.unique()

array(['State', 'Carlow', 'Dublin', 'Dún Laoghaire-Rathdown', 'Fingal',
       'South Dublin', 'Kildare', 'Kilkenny', 'Laois', 'Longford',
       'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford', 'Wicklow',
       'Clare', 'Cork', 'Kerry', 'Limerick', 'Tipperary', 'Waterford',
       'Galway', 'Leitrim', 'Mayo', 'Roscommon', 'Sligo', 'Cavan',
       'Donegal', 'Monaghan'], dtype=object)

**unifying columns names with the SAP dataframe**

In [171]:
df_aux = create_region_column(df_aux, df_reg)
df_aux.head()

,Statistic Label,CensusYear,County,UNIT,VALUE,Region
0,Private households,2011,State,Number,1654208.0,State
1,Private households,2011,Carlow,Number,19436.0,South-East
2,Private households,2011,Dublin,Number,208008.0,Dublin
3,Private households,2011,Dún Laoghaire-Rathdown,Number,75819.0,Dublin
4,Private households,2011,Fingal,Number,93146.0,Dublin


In [172]:
df_aux["County"] = df_aux["Region"]
df_aux = df_aux.rename(columns={'County': 'Name of Region'})
df_aux = df_aux.drop(columns=["Region"])

In [180]:
df_aux['Name of Region'].unique()

array(['State', 'South-East', 'Dublin', 'Mid-East', 'Midlands', 'Border',
       'Mid-West', 'South-West', 'West'], dtype=object)

Here we hava 'State', reflecting value for the country and for better readability and unification will rename it in 'Ireland'

In [181]:
df_aux = df_aux.replace('State', 'Ireland')

**Splitting Values foe statistic labels**
* removing uneecessary data with 'Average number of persons in private households' label
* create 2 additional column with names from according label value name. I.e. 'Private households' and 'Persons in private households'. 
* Inserting according value to each row

In [182]:
df_aux = df_aux.drop(df_aux[(df_aux["Statistic Label"] == 'Average number of persons in private households')].index)
df_aux['Statistic Label'].unique()

array(['Private households', 'Persons in private households'],
      dtype=object)

In [175]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Statistic Label  186 non-null    object 
 1   CensusYear       186 non-null    int64  
 2   Name of Region   186 non-null    object 
 3   UNIT             186 non-null    object 
 4   VALUE            186 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 7.4+ KB


In [183]:
#Export DataFrame to a csv File
df_aux.to_csv("FY004B_c.csv", index = False)